In [40]:
#Importaciones
from functions import *
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
#Cargando base de datos
url_robberies = ("https://github.com/LuisHRF/Home-Insurance-Study-Project-/raw/main/01002%20(1).xlsx")
data_raw_robberies = pd.read_excel(url_robberies)

In [ ]:
data_clean = data_raw_robberies.dropna()
data_clean 

In [ ]:
# Ver los valores únicos de la primera columna
unique_first_column = data_clean.iloc[:, 0].unique()
unique_first_column

In [ ]:
block_sizes = [55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55]
start = 1
year_ranges = {}

for i, size in enumerate(block_sizes):
    end = start + size - 1
    year = 2023 - i
    year_ranges[(start, end)] = year
    start = end + 2  

print(year_ranges)

In [6]:
def defi_years_per_block(data_frame, year_range):

    data_frame_cp = data_frame.copy()

    for (first_row, last_row), year in year_range.items():
        data_frame_cp.loc[first_row:last_row, 'Year'] = year

    return data_frame_cp

In [7]:
data_clean_years = defi_years_per_block(data_clean, year_ranges)

In [ ]:
data_clean_years.tail(10)

In [ ]:
# Limpieza de Encabezados
data_clean_years.columns = data_clean_years.columns.str.lower().str.replace(" ", "_")
data_clean_years = data_clean_years.rename(columns={
    "_": "Province"
})
data_clean_years

In [10]:
columns_titles = ["1._contra_las_personas","1.2.-lesiones","5.1.-hurtos","5.2.-robos_con_fuerza_en_las_cosas","5.2.1.-robos_con_fuerza_en_las_cosas_en_el_interior_de_vehículos","5.2.2.-robos_con_fuerza_en_viviendas","5.2.3.-robos_con_fuerza_en_establecimientos","5.3.-robos_con_violencia_o_intimidación","5.3.1.-robos_con_violencia_en_vía_pública","5.3.2.-robos_con_violencia_en_viviendas","5.3.3.-robos_con_violencia_en_establecimientos"]
translations = {"1._contra_las_personas": "Assault",
                "1.2.-lesiones" :"Injuries",
                "5.1.-hurtos":"small robberies",
                "5.2.-robos_con_fuerza_en_las_cosas":"robs_with_force",
                "5.2.1.-robos_con_fuerza_en_las_cosas_en_el_interior_de_vehículos":"robs_force_vehicles",
                "5.2.2.-robos_con_fuerza_en_viviendas":"robs_force_homes",
                "5.2.3.-robos_con_fuerza_en_establecimientos":"robs_force_stores",
                "5.3.-robos_con_violencia_o_intimidación":"robs_violence_intimidation",
                "5.3.1.-robos_con_violencia_en_vía_pública":"robs_violence_publicways",
                "5.3.2.-robos_con_violencia_en_viviendas":"robs_violence_homes",
                "5.3.3.-robos_con_violencia_en_establecimientos":"robs_violence_stores"
}

In [ ]:
def traducir_columnas(columns_titles, translations):
    return [translations.get(col, col) for col in columns_titles]
print(traducir_columnas(columns_titles, translations))

In [12]:
# Suponiendo que tu DataFrame se llama data_clean_years
data_clean_years.columns = traducir_columnas(data_clean_years.columns, translations)

In [ ]:
print(data_clean_years)

In [ ]:
# Eliminar la columna sin nombre (usualmente aparece como "Unnamed: 0")
data_clean_years = data_clean_years.drop(columns=[col for col in data_clean_years.columns if "Unnamed" in col])
data_clean_years

In [ ]:
# Restablecer el índice en 10 resultados
data_clean_years.reset_index(drop=True, inplace=True)
print(data_clean_years.to_string(index=False))

In [ ]:
data_clean_years.head(100)

In [17]:
'''
def verify_and_switch_datatypes(data_clean_years, column_name, assigned_types):
    column_name = data_clean_years[column_name]
    currently_type = data_clean_years[column_name].dtype
    assigned_types = {'Province': str,
                      'Assault': int,
                      'physical injuries':int,
                      'small_robberies':int,
                      'robberies_with_force':int,
                      'robberies_force_vehicles': int, 
                      'hard_robbvehicles':int, 
                      'robs_force_homes':int, 
                      'robs_force_establishments':int, 
                      'robs_violence_intimidation':int,
                      'robs_violence_publicways':int,
                      'robs_violence_homes':int, 
                      'robs_violence_establishments':int,
                      'year':int
                      }
    for column_name, assigned_types in assigned_types.items():
        if currently_type != assigned_types:
            try:
                data_clean_years[column_name] = data_clean_years[column_name].astype(assigned_types)
                print(f"Column '{column_name}' converted from {currently_type} a {assigned_types}.")
            except Exception as e:
                print(f"It was no possible to switch the '{column_name}' type into {assigned_types}: {e}")
        else:
            print(f"Column '{column_name}' is currently right typed as ({currently_type}).")
    
    return data_clean_years
'''

'\ndef verify_and_switch_datatypes(data_clean_years, column_name, assigned_types):\n    column_name = data_clean_years[column_name]\n    currently_type = data_clean_years[column_name].dtype\n    assigned_types = {\'Province\': str,\n                      \'Assault\': int,\n                      \'physical injuries\':int,\n                      \'small_robberies\':int,\n                      \'robberies_with_force\':int,\n                      \'robberies_force_vehicles\': int, \n                      \'hard_robbvehicles\':int, \n                      \'robs_force_homes\':int, \n                      \'robs_force_establishments\':int, \n                      \'robs_violence_intimidation\':int,\n                      \'robs_violence_publicways\':int,\n                      \'robs_violence_homes\':int, \n                      \'robs_violence_establishments\':int,\n                      \'year\':int\n                      }\n    for column_name, assigned_types in assigned_types.items():\n 

In [ ]:
def verify_and_switch_datatypes(data_clean_years, assigned_types):
    # Itera sobre cada columna y su tipo asignado en el diccionario
    for column_name, expected_type in assigned_types.items():
        # Obtiene el tipo actual de la columna
        currently_type = data_clean_years[column_name].dtype
        
        if currently_type != expected_type:
            try:
                # Convierte la columna al tipo esperado
                data_clean_years[column_name] = data_clean_years[column_name].astype(expected_type)
                print(f"Column '{column_name}' converted from {currently_type} to {expected_type}.")
            except Exception as e:
                print(f"It was not possible to switch the '{column_name}' type to {expected_type}: {e}")
        else:
            print(f"Column '{column_name}' is currently correctly typed as ({currently_type}).")
    
    return data_clean_years

# Ejemplo de cómo aplicar la función
assigned_types = {
    'Province': str,
    'Assault': int,
    'Injuries': int,
    'small robberies': int,
    'robs_with_force': int,
    'robs_force_vehicles': int,  
    'robs_force_homes': int,
    'robs_force_stores': int,
    'robs_violence_intimidation': int,
    'robs_violence_publicways': int,
    'robs_violence_homes': int, 
    'robs_violence_stores': int,
    'year': int
}

# Supongamos que `data_clean_years` es tu DataFrame
data_clean_years = verify_and_switch_datatypes(data_clean_years, assigned_types)

In [19]:
def convert_year_into_datetime(data_clean_years):
    data_clean_years['year'] = pd.to_datetime(data_clean_years['year'].astype(str) + '-01-01')

In [ ]:
data_clean_years

In [ ]:
# SUMAR TODOS LOS DELITOS DE TODOS LOS AÑOS PARA CADA PROVINCIA

delitos_columns = [
    'Assault', 'Injuries', 'small robberies', 'robs_with_force',
    'robs_force_vehicles', 'robs_force_homes', 'robs_force_stores',
    'robs_violence_intimidation', 'robs_violence_publicways',
    'robs_violence_homes', 'robs_violence_stores'
]

# Agrupar y sumar
df_suma_delitos = data_clean_years.groupby('Province')[delitos_columns].sum().reset_index()

print(df_suma_delitos)

In [23]:
# df_suma_delitos.to_excel('suma_delitos_por_provincia.xlsx', index=False)

In [ ]:
df_suma_delitos['Total'] = df_suma_delitos.iloc[:, 1:].sum(axis=1)

# Ver el resultado
print(df_suma_delitos[['Province', 'Total']])

### VAMOS CON LAS TENDENCIAS

In [ ]:
# Tenemos que agrupar todos los delitos por año y provincia

columns_to_sum = ['Assault', 'Injuries', 'small robberies', 'robs_with_force',
                  'robs_force_vehicles', 'robs_force_homes', 'robs_force_stores',
                  'robs_violence_intimidation', 'robs_violence_publicways',
                  'robs_violence_homes', 'robs_violence_stores']

# Creamos una nueva columna que sume todos los delitos para cada provincia y año
data_clean_years['total_crimes'] = data_clean_years[columns_to_sum].sum(axis=1)

# Visualizamos el DataFrame con la nueva columna
print(data_clean_years[['Province', 'year', 'total_crimes']])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Aseguramos que el DataFrame esté ordenado por provincia y año
df_sorted = data_clean_years[['Province', 'year', 'total_crimes']].sort_values(by=['Province', 'year'])

# Configuración del tamaño de la gráfica
plt.figure(figsize=(12, 8))

# Graficamos las tendencias de cada provincia
sns.lineplot(data=df_sorted, x='year', y='total_crimes', hue='Province', marker='o', legend=False)

# Ajustamos el título y las etiquetas
plt.title('Tendencia de delitos totales por provincia a lo largo de los años', fontsize=16)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Delitos Totales', fontsize=12)

# Mejoramos la visualización de los ticks en el eje X
plt.xticks(rotation=45)

# Mostramos la gráfica
plt.tight_layout()
plt.show()

In [27]:
# Filtrar las provincias con menos incremento tendencial

In [28]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Inicializamos un diccionario para almacenar la tasa de crecimiento de cada provincia
crecimiento = {}

# Iteramos sobre cada provincia
for provincia in data_clean_years['Province'].unique():
    df_provincia = data_clean_years[data_clean_years['Province'] == provincia]
    
    # Preparamos los datos para la regresión
    X = df_provincia['year'].values.reshape(-1, 1)
    y = df_provincia['total_crimes'].values
    
    # Realizamos la regresión lineal
    modelo = LinearRegression()
    modelo.fit(X, y)
    
    # Almacenamos la pendiente (tasa de crecimiento) en el diccionario
    crecimiento[provincia] = modelo.coef_[0]

# Convertimos el diccionario en un DataFrame para facilitar la manipulación
df_crecimiento = pd.DataFrame(list(crecimiento.items()), columns=['Province', 'Growth Rate'])

# Ordenamos las provincias por tasa de crecimiento y seleccionamos las más altas
top_provincias = df_crecimiento.sort_values(by='Growth Rate', ascending=False).head(10)['Province'].tolist()

In [29]:
# Filtramos el DataFrame para incluir solo las provincias con mayor crecimiento
df_filtrado_crecimiento = data_clean_years[data_clean_years['Province'].isin(top_provincias)]

# Ordenamos el DataFrame filtrado por provincia y año
df_filtrado_crecimiento_sorted = df_filtrado_crecimiento[['Province', 'year', 'total_crimes']].sort_values(by=['Province', 'year'])

In [ ]:
# Configuración del tamaño de la gráfica
plt.figure(figsize=(12, 8))

# Graficar las tendencias de las provincias con mayor crecimiento
sns.lineplot(data=df_filtrado_crecimiento_sorted, x='year', y='total_crimes', hue='Province', marker='o', palette='tab10')

# Ajustar el título y las etiquetas
plt.title('Tendencia de delitos totales en provincias con mayor crecimiento tendencial', fontsize=16)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Delitos Totales', fontsize=12)

# Mejorar la visualización de los ticks en el eje X
plt.xticks(rotation=45)

# Mostrar la gráfica
plt.tight_layout()
plt.show()

In [31]:
# Incremento porcentual entre el primer y el último año

In [ ]:
# Agrupar los datos por provincia y obtener el total de delitos del primer y último año
df_grouped = data_clean_years.groupby(['Province', 'year'])['total_crimes'].sum().reset_index()

# Encontrar el primer y el último año para cada provincia
df_first_last = df_grouped.groupby('Province').agg(
    first_year_total=('total_crimes', 'first'),
    last_year_total=('total_crimes', 'last')
).reset_index()

# Calcular el incremento porcentual
df_first_last['percentage_increase'] = ((df_first_last['first_year_total'] - df_first_last['last_year_total']) /
                                        df_first_last['last_year_total']) * 100

# Ordenar las provincias por incremento porcentual
df_sorted_increase = df_first_last.sort_values(by='percentage_increase', ascending=False)

# Mostrar el resultado
print(df_sorted_increase[['Province', 'first_year_total', 'last_year_total', 'percentage_increase']])

In [ ]:
# Configuración del tamaño de la gráfica
plt.figure(figsize=(12, 8))

# Graficar el incremento porcentual de delitos por provincia
sns.barplot(data=df_first_last, x='percentage_increase', y='Province', palette='viridis')

# Ajustar el título y las etiquetas
plt.title('Incremento Porcentual de Delitos por Provincia', fontsize=16)
plt.xlabel('Incremento Porcentual (%)', fontsize=12)
plt.ylabel('Provincia', fontsize=12)

# Mejorar la visualización de los ticks en el eje X
plt.tight_layout()

# Mostrar la gráfica
plt.show()

In [ ]:
# Agrupar por provincia y año y calcular la media de cada delito
df_mean = data_clean_years.groupby(['Province', 'year'])[['Assault', 'Injuries', 'small robberies', 
                                            'robs_with_force', 'robs_force_vehicles', 
                                            'robs_force_homes', 'robs_force_stores', 
                                            'robs_violence_intimidation', 'robs_violence_publicways', 
                                            'robs_violence_homes', 'robs_violence_stores']].mean().reset_index()

# Mostrar el resultado
print(df_mean.head())

In [ ]:
# Configuración del tamaño de la gráfica
plt.figure(figsize=(14, 10))

# Graficar las medias de delitos por provincia
df_melted = df_mean.melt(id_vars='Province', var_name='Type', value_name='Average')

# Graficar
sns.barplot(data=df_melted, x='Average', y='Province', hue='Type', palette='tab10')

# Ajustar el título y las etiquetas
plt.title('Media de Delitos por Provincia', fontsize=16)
plt.xlabel('Media de Delitos', fontsize=12)
plt.ylabel('Provincia', fontsize=12)

# Mostrar la gráfica
plt.tight_layout()
plt.show()

In [ ]:
# USELESS TRASH
'''
print(data_clean_years.columns)
data_clean_years.columns = data_clean_years.columns.str.strip()
if 'Assault' not in data_clean_years.columns:
    print("La columna 'Assault' no se encuentra en el DataFrame.")
else:
    print("La columna 'Assault' está presente.")
'''
#data_formated_year = convert_year_into_datetime(data_clean_years)
#df.columns = df.columns.str.lower().str.replace(" ","_").str.replace(".","_").str.replace(":","").str.strip()
#df.rename(columns={"species_" : "species", "unnamed_11" : "death"}, inplace= True)

'''
MAPA COLOREADO POR PROVINCIAS
# Cargar el shapefile de las provincias
gdf_provincias = gpd.read_file('path_to_shapefile.shp')  # Reemplaza con el camino a tu shapefile

# Cargar el DataFrame con los datos de delitos
df_delitos = pd.read_excel('suma_delitos_por_provincia.xlsx')  # Reemplaza con el camino a tu archivo Excel

# Asegúrate de que los nombres de las provincias en ambos DataFrames coincidan
# Puedes necesitar ajustar los nombres o hacer una correspondencia entre ellos

# Unir los datos de delitos con el GeoDataFrame
gdf_provincias = gdf_provincias.merge(df_delitos, left_on='province_column_in_shapefile', right_on='Province')  # Ajusta los nombres de las columnas

# Crear el mapa
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf_provincias.plot(column='Assault',  # Reemplaza con la columna que desees visualizar
                    cmap='OrRd',  # Elige un esquema de colores
                    linewidth=0.8,
                    ax=ax,
                    edgecolor='0.8',
                    legend=True)

ax.set_title('Número de Delitos por Provincia en España')
ax.set_axis_off()

plt.show()
'''